In [1]:
# Importing Packages

from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import get_requests as gr
import time
import pprint
import json

In [ ]:
# Objects Used By Other Functions (Unfinished)

commodities = dict() # GET v2/commodity-types.json
crop_stages = list() # GET v2/crop-stages.json?commodityTypeId={commodityTypeId}
crop_types = dict() # GET v2/crop-types.{ext}?includePrevious={includePrevious}&commodityTypeId={commodityTypeId}
crop_type_data = list() # GET v2/crop-types/{cropTypeId}.{ext}
ranches = dict() # GET v2/ranches.{ext}
ranch_crop_types_list = list() # GET v2/ranches/{ranchGuid}/commodity-types/{commodityTypeId}/crop-types.{ext}?shouldFilterUnassociated={shouldFilterUnassociated}
ranch_plantings_list = list() # GET v2/ranches/{ranchGuid}/plantings.{ext}?active={active}&commodityTypeId={commodityTypeId}
ranch_details_list = list() # GET v2/ranches/{ranchGuid}.json
ranch_permissions_list_global = list() # GET v2/ranches/{ranchGuid}/permissions.{ext}
lots = list() # GET v2/ranches/{ranchGuid}/lots.json
lots_details_list = list() # GET v2/lots/{lotId}.json
wells = list() # GET v2/ranches/{id}/ranch-wells.{ext}
wells_details_list_global = list() # GET v2/wells/{wellId}.json
regions = dict() # GET v2/regions.{ext}
soil_types = dict() # GET v2/soil-types.{ext}
user_details_list = list() # GET v2/user-detail.{ext}
tissue_sample_locations = dict() # GET v2/tissue-sample-locations.{ext}
tissue_sample_nutrients_list = list() # GET v2/tissue-sample-nutrients.{ext}
associated_users_list = list() # GET v2/ranches/{ranchGuid}/users.json
weather_apis = dict() # GET v2/weatherapis.{ext}
ranch_weather_stations_list = list() # GET v2/ranches/{ranchGuid}/weather-stations.{ext}
weather_stations = dict() # GET v2/weather-stations.{ext}?activeOnly={activeOnly}&ranchGuid={ranchGuid}&suppressZeroET={suppressZeroET}

In [2]:
# Function To Save Response To CSV
def save_to_csv(json, csv_name):
    df = pd.DataFrame(json)
    df.to_csv(f"./csv/{csv_name}.csv", index=False)
    
# Function To Save Response To JSON
def save_to_json(data, json_name):
    with open(f"./json/{json_name}.json", 'w') as file:
        json.dump(data, file, indent=4)

In [3]:
# Function To Flatten A JSON

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if isinstance(x, dict):
            for a in x:
                flatten(x[a], name + a + '_')
        elif isinstance(x, list):
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [8]:
# GET v2/commodity-types.json -> commodity-types.csv

def func_get_commodity_types():
    
    # GET Request For Commodity Types
    commodities = gr.get_commodity_types()
    
    # If Result Is A Response Code, Return
    if isinstance(commodities, int):
        return
    else:
        
        # Save Result As JSON And CSV
        save_to_json(commodities, "commodity-types")
        save_to_csv(commodities, "commodity-types")

In [ ]:
# GET v2/crop-stages.json?commodityTypeId={commodityTypeId} -> crop-stages.csv

def func_get_crop_stages():
    
    # Using Commodities JSON To Augment Crop Stages JSON
    for commodity in commodities:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        commodity_id = commodity["Id"]
        commodity_name = commodity["Name"]
    
        # GET Request For Crop Stages For Current Commodity
        crop_stage = gr.get_crop_stages(commodity_id)
        
        # If Result Is A Response Code, Continue
        if isinstance(crop_stage, int):
            continue
        else:
            # Adding Commodity ID and Commodity Name To Each Crop Stage
            for stage in crop_stage:
                stage["CommodityId"] = commodity_id
                stage["CommodityName"] = commodity_name
    
            # Adding Crop Stages To List Of Crop Stages
            crop_stages += crop_stage
        
            # For Progress Purposes
            print(crop_stage)
    
        # To Not Overload The Server
        time.sleep(4)

    # Save Result As JSON And CSV
    save_to_json(crop_stages, "crop-stages")
    save_to_csv(crop_stages, "crop-stages")

In [10]:
# GET v2/crop-types.{ext}?includePrevious={includePrevious}&commodityTypeId={commodityTypeId}
# -> crop-types.csv

def func_get_crop_types():
    
    # GET Request For Crop Types
    crop_types = gr.get_crop_types()
    
    # If Result Is A Response Code, Return
    if isinstance(crop_types, int):
        return
    else:
    
        # Save Result As JSON And CSV
        save_to_json(crop_types, "crop-types")
        save_to_csv(crop_types, "crop-types")

In [ ]:
# GET v2/crop-types/{cropTypeId}.{ext} -> crop-type-data.csv

def func_get_crop_type_data():
    
    # Using Crop Types JSON To Get Crop Type Data
    for type in crop_types:
        
        # Using This Field To Retrieve Resulting JSON
        crop_type_id = type["Id"]
        
        # GET Request For Crop Type By ID
        ctd = gr.get_crop_type_by_id(crop_type_id)
        
        # If Result Is A Response Code, Continue
        if isinstance(ctd, int):
            continue
        else:
            
            # Flatten JSON For CSV Purposes
            ctd_flat = flatten_json(ctd)
            
            # Append Flattened JSON To Crop Type Data List
            crop_type_data.append(ctd_flat)
            
            # For Progress Purposes
            print(ctd_flat)
            
        # To Not Overload The Server
        time.sleep(4)

    # Save Result As JSON And CSV
    save_to_json(crop_type_data, 'crop-type-data')
    save_to_csv(crop_type_data, 'crop-type-data')

In [ ]:
# GET v2/ranches.{ext} -> ranches.csv

def func_get_ranches():
    
    # GET Request For Ranches
    ranches = gr.get_ranches()
    
    # If Result Is A Response Code, Return
    if isinstance(ranches, int):
        return
    else:
        
        # Save Result As JSON And CSV
        save_to_json(ranches, 'ranches')
        save_to_csv(ranches, 'ranches')

In [ ]:
# GET v2/ranches/{ranchGuid}/commodity-types/{commodityTypeId}/crop-types.{ext}?shouldFilterUnassociated={shouldFilterUnassociated}
# -> ranch-{ranch_id}-crop-types.csv

def func_get_ranch_crop_types():
    
    # Going Through User's Ranches
    for ranch in ranches:
        ranch_crop_types = list()
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # Going Through Each Of The Commodities
        for commodity in commodities:
            
            # Using This Field TO Retrieve + Augment Resulting JSON
            commodity_id = commodity["Id"]
            
            # GET Request For Associated Crop Types Of Current Ranch
            associated_crop_types = gr.get_crop_types_of_ranch(ranch_guid, commodity_id)
            
            # If Result Is A Response Code, Add Dictionary
            if isinstance(associated_crop_types, int):
                associated_crop_types = dict()
                associated_crop_types["Ranch_External_GUID"] = ranch_guid
                associated_crop_types["CommodityId"] = commodity_id       
            else:
                
                # Go Through Each Crop Type And Augment The Fields Using The Commodity ID and Ranch GUID
                for type in associated_crop_types:
                    type["Ranch_External_GUID"] = ranch_guid
                    type["CommodityId"] = commodity_id
                
                # Flatten JSON For CSV Purposes
                associated_crop_types = flatten_json(associated_crop_types)
                
            # For Progress Purposes
            print(associated_crop_types)
            
            # Append Associated Crop Types To Ranch Crop Types
            ranch_crop_types.append(associated_crop_types)
            
            # To Not Overload The Server
            time.sleep(4)
        
        # Append To Global Variable And Save As JSON And CSV
        ranch_crop_types_list.append(ranch_crop_types)
        save_to_json(ranch_crop_types, f"ranch-{ranch_id}-crop-types")
        save_to_csv(ranch_crop_types, f"ranch-{ranch_id}-crop-types")

In [ ]:
# GET v2/ranches/{ranchGuid}/plantings.{ext}?active={active}&commodityTypeId={commodityTypeId}
# -> ranch-{ranch_id}-plantings.csv

def func_get_ranch_plantings():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Plantings
        plantings = gr.get_ranch_plantings(ranch_guid)
        
        # If Result Is A Response Code, Enter Empty Dictionary
        if isinstance(plantings, int):
            plantings = dict()
        else:
            
            # Iterate Through Plantings, Augment Them, And Flatten Them
            for i in range(0, len(plantings)):
                plantings[i]["Ranch_External_GUID"] = ranch_guid
                plantings[i] = flatten_json(plantings[i])
        
        # For Progress Purposes
        print(plantings)
        
        # Append To Global Variable And Save As JSON And CSV
        ranch_plantings_list.append(plantings)
        save_to_json(plantings, f"ranch-{ranch_id}-plantings")
        save_to_csv(plantings, f"ranch-{ranch_id}-plantings")
        
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/ranches/{ranchGuid}.json -> {ranch_id}.csv

def func_get_ranch_details():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Details
        ranch_info = gr.get_ranch_details(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_info, int):
            continue
        else:
            
            # Augment Result
            ranch_info["Id"] = ranch_id
            ranch_info["Ranch_External_GUID"] = ranch_guid
            
            # For CSV Purposes
            ranch_info_list = list()
            ranch_info_list.append(ranch_info)
            
            # For Progress Purposes
            print(ranch_info_list)
            
            # Append To Global Variable And Save AS JSON And CSV
            ranch_details_list.append(ranch_info_list)
            save_to_json(ranch_info_list, f"{ranch_id}")
            save_to_csv(ranch_info_list, f"{ranch_id}")
            
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/ranches/{ranchGuid}/permissions.{ext} -> ranch-{ranch_id}-permissions.csv

def func_get_ranch_permissions():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Permissions
        ranch_permissions = gr.get_ranch_planting_permissions(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_permissions, int):
            continue
        else:
            
            # For CSV Purposes
            ranch_permissions_list = list()
            ranch_permissions_list.append(ranch_permissions)
            
            # For Progress Purposes
            print(ranch_permissions_list)
            
            # Append To Global Variable And Save As JSON And CSV
            ranch_permissions_list_global.append(ranch_permissions_list)
            save_to_json(ranch_permissions_list, f"ranch-{ranch_id}-permissions")
            save_to_csv(ranch_permissions_list, f"ranch-{ranch_id}-permissions")

        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/ranches/{ranchGuid}/lots.json -> ranch-{ranch_id}-lots.csv

def func_get_planting_areas():

    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Planting Areas
        ranch_planting_areas = gr.get_planting_areas(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_planting_areas, int):
            continue
        else:
            
            # Augmenting Each Planting Area
            for planting_area in ranch_planting_areas:
                planting_area["Ranch_External_GUID"] = ranch_guid
        
        # For Progress Purposes
        print(ranch_planting_areas)
        
        # Append To Global Variable And Save As JSON And CSV
        lots.append(ranch_planting_areas)
        save_to_json(ranch_planting_areas, f'ranch-{ranch_id}-lots')
        save_to_csv(ranch_planting_areas, f'ranch-{ranch_id}-lots')
        
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/lots/{lotId}.json -> lot-{lotId}.csv

def func_get_planting_area_details():
    
    # Going Through User's Lots
    for lot in lots:
        
        # Going Through Each Area Within Lot
        for area in lot:
            
            # Using This Field To Retrieve Resulting JSON
            lot_id = area["Id"]
            
            # GET Request For Lot Details
            planting_area_details = gr.get_planting_area_details(lot_id)
            
            # If Result Is A Response Code, Continue
            if isinstance(planting_area_details, int):
                continue
            else:
                
                # Flattening JSON For CSV Purposes
                planting_area_details_flat = flatten_json(planting_area_details)
                
                # More Altering For CSV Purposes
                planting_area_details_list = list()
                planting_area_details_list.append(planting_area_details_flat)
                
                # For Progress Purposes
                print(planting_area_details_list)
                
                # Append To Global Variable And Save As JSON And CSV
                lots_details_list.append(planting_area_details_list)
                save_to_json(planting_area_details_list, f"lot-{lot_id}")
                save_to_csv(planting_area_details_list, f"lot-{lot_id}")
                
            # To Not Overload The Server
            time.sleep(4)
                

In [ ]:
# GET v2/ranches/{id}/ranch-wells.{ext} -> ranch-{ranch_id}-wells.csv

def func_get_ranch_wells():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Wells
        ranch_wells = gr.get_wells(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_wells, int):
            continue
        else:
            
            # For Progress Purposes
            print(ranch_wells)
            
            # Appending To Global Variable
            for ranch_well in ranch_wells:
                wells.append(ranch_well)
            
            # Save As JSON And CSV
            save_to_json(ranch_wells, f"ranch-{ranch_id}-wells")
            save_to_csv(ranch_wells, f"ranch-{ranch_id}-wells")
            
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/wells/{wellId}.json -> well-{wellId}.csv

def func_get_well_details():
    
    # Going Through User's Wells
    for well in wells:
        
        # Using This Field To Retrieve Resulting JSON
        well_id = well["Id"]
        
        # GET Request For Well Details
        well_details = gr.get_well_details(well_id)
        
        # If Result Is A Response Code, Continue
        if isinstance(well_details, int):
            continue
        else:
            
            # For CSV Purposes
            well_details_list = list()
            well_details_list.append(well_details)
            
            # For Progress Purposes
            print(well_details_list)
            
            # Append To Global Variable And Save As JSON And CSV
            wells_details_list_global.append(well_details_list)
            save_to_json(well_details_list, f"well-{well_id}")
            save_to_csv(well_details_list, f"well-{well_id}")
        
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/regions.{ext} -> regions.csv

def func_get_regions():
    
    # GET Request For Regions
    regions = gr.get_regions()
    
    # If Result Is A Response Code, Return
    if isinstance(regions, int):
        return
    else:
        
        # Save As JSON And CSV
        save_to_json(regions, "regions")
        save_to_csv(regions, "regions")

In [ ]:
# GET v2/soil-types.{ext} -> soil-types.csv

def func_get_soil_types():
    
    # GET Request For Soil Types
    soil_types = gr.get_soil_types()
    
    # If Result Is A Response Code, Return
    if isinstance(soil_types, int):
        return
    else:
        
        # Save As JSON And CSV
        save_to_json(soil_types, "soil-types")
        save_to_csv(soil_types, "soil-types")

In [ ]:
# GET v2/user-detail.{ext} -> logged-in-user.csv

def func_get_user_details():
    
    # GET Request For Logged In User Details
    user_details = gr.get_user_details()
    
    # If Result Is A Response Code, Return
    if isinstance(user_details, int):
        return
    else:
        
        # Flattening JSON For CSV Purposes
        user_details_flat = flatten_json(user_details)

        # Append To Global Variable And Save As JSON And CSV
        user_details_list.append(user_details_flat)
        save_to_json(user_details_list, "logged-in-user")
        save_to_csv(user_details_list, "logged-in-user")

In [ ]:
# GET v2/tissue-sample-locations.{ext} -> tissue-sample-locations.csv

def func_get_tissue_sample_locations():
    
    # GET Request For Tissue Sample Locations
    tissue_sample_locations = gr.get_tissue_sample_locations()
    
    # If Result Is A Response Code, Return
    if isinstance(tissue_sample_locations, int):
        return
    else:
        
        # Save As JSON And CSV
        save_to_json(tissue_sample_locations, "tissue_sample_locations")
        save_to_csv(tissue_sample_locations, "tissue_sample_locations")

In [ ]:
# GET v2/tissue-sample-nutrients.{ext} -> tissue-sample-nutrients.csv

def func_get_tissue_sample_nutrients():
    
    # GET Request For Tissue Sample Nutrients
    tissue_sample_nutrients = gr.get_tissue_sample_nutrients()
    
    # If Result Is A Response Code, Return
    if isinstance(tissue_sample_nutrients, int):
        return
    else:
    
        # Flattening JSON For CSV Purposes
        tissue_sample_nutrients_flat = flatten_json(tissue_sample_nutrients)
        
        # Append To Global Variable And Save As JSON And CSV
        tissue_sample_nutrients_list.append(tissue_sample_nutrients_flat)
        save_to_json(tissue_sample_nutrients_list, "tissue_sample_nutrients")
        save_to_csv(tissue_sample_nutrients_list, "tissue_sample_nutrients")

In [ ]:
# GET v2/ranches/{ranchGuid}/users.json -> ranch-{ranch_id}-users.csv

def func_get_ranch_users():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Augment + Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Associated Users
        associated_users = gr.get_associated_users(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(associated_users, int):
            continue
        else:
            
            # For Progress Purposes
            print(associated_users)
            
            # Append To Global Variable And Save As JSON And CSV
            associated_users_list.append(associated_users)
            save_to_json(associated_users, f"{ranch_id}-users")
            save_to_csv(associated_users, f"{ranch_id}-users")
        
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/weatherapis.{ext} -> weatherapis.csv

def func_get_weather_service_records():
    
    # GET Request For Weather Service Records
    weather_apis = gr.get_weather_service_records()
    
    # If Result Is A Respone Code, Return
    if isinstance(weather_apis, int):
        return
    else:
        
        # Save As JSON And CSV
        save_to_json(weather_apis, "weatherapis")
        save_to_csv(weather_apis, "weatherapis")

In [ ]:
# GET v2/ranches/{ranchGuid}/weather-stations.{ext} -> ranch-{ranch_id}-weather-stations.csv

def func_get_weather_stations_by_ranch():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Ranch Weather Stations
        ranch_weather_stations = gr.get_weather_stations_by_ranch(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_weather_stations, int):
            continue
        else:
            
            # Flatten Each Weather Station For CSV Purposes
            for i in range(0, len(ranch_weather_stations)):
                ranch_weather_stations[i] = flatten_json(ranch_weather_stations[i])
                
            # For Progress Purposes
            print(ranch_weather_stations)
            
            # Append To Global Variable And Save As JSON And CSV
            ranch_weather_stations_list.append(ranch_weather_stations)
            save_to_json(ranch_weather_stations, f"ranch-{ranch_id}-weather-stations")
            save_to_csv(ranch_weather_stations, f"ranch-{ranch_id}-weather-stations")
        
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/weather-stations.{ext}?activeOnly={activeOnly}&ranchGuid={ranchGuid}&suppressZeroET={suppressZeroET}
# -> weather-stations.csv

def func_get_all_weather_stations():
    
    # GET Request For Weather Stations
    weather_stations = gr.get_all_weather_stations()
    
    # If Result Is A Response Code, Return
    if isinstance(weather_stations, int):
        return
    else:
        
        # Save As JSON And CSV
        save_to_json(weather_stations, "weather-stations")
        save_to_csv(weather_stations, "weather-stations")

In [ ]:
# GET v2/ranches/{ranchGuid}/users/{userId}/permissions.json -> ranch-{ranch_id}-users-permissions.csv

def func_get_ranch_users_permissions():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using These Fields To Retrieve + Augment Resulting JSON
        ranch_id = ranch["Id"]
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # Return Variable With Ranch User Permissions
        ranch_user_permissions_list = list()
        
        # Opening Ranch Users JSON File
        with open(f"./json/{ranch_id}-users.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each User In The Ranch
            for item in data:
                user_id = item["UserId"]
                
                # GET Request For User's Ranch Permissions For Given Ranch
                ranch_user_permissions = gr.get_ranch_permissions(ranch_guid, user_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(ranch_user_permissions, int):
                    continue
                else:
                    # Augment JSON Response
                    ranch_user_permissions["ranchGuid"] = ranch_guid
                    ranch_user_permissions["userId"] = user_id
                    
                    # For CSV Purposes
                    ranch_user_permissions_flat = flatten_json(ranch_user_permissions)
                    ranch_user_permissions_list.append(ranch_user_permissions_flat)
                
                # To Not Overload The Server 
                time.sleep(4)
                    
            # For Progress Purposes
            print(ranch_user_permissions_list)
                    
            # Save As JSON And CSV
            save_to_json(ranch_user_permissions_list, f"ranch-{ranch_id}-users-permissions")
            save_to_csv(ranch_user_permissions_list, f"ranch-{ranch_id}-users-permissions")

In [ ]:
# GET v2/maturities.{ext} -> maturities.csv

def func_get_maturities():
    
    # GET Request For Maturities
    maturities = gr.get_maturities()
    
    # Save Result As JSON And CSV
    save_to_json(maturities, "maturities")
    save_to_csv(maturities, "maturities")

In [ ]:
# GET v2/microsprinklers.{ext} -> microsprinklers.csv

def func_get_microsprinklers():
    
    # GET Request For Microsprinklers
    microsprinklers = gr.get_microsprinklers()

    # Iterating Through Response And Flattening Each Item
    for i in range(0, len(microsprinklers)):
        microsprinklers[i] = flatten_json(microsprinklers[i])
    
    # Save Result As JSON And CSV
    save_to_json(microsprinklers, "microsprinklers")
    save_to_csv(microsprinklers, "microsprinklers")

In [ ]:
# GET v2/plantings/{id}.{ext} -> planting-{plantingId}.csv

def func_get_planting_details():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrive Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Details
                planting_details = gr.get_planting_details(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_details, int):
                    continue
                else:
                    # For CSV Purposes
                    planting_details_flat = flatten_json(planting_details)
                    planting_details_list = list()
                    planting_details_list.append(planting_details_flat)
                    
                    # For Progress Purposes
                    print(planting_details_list)
                    
                    # Save As JSON And CSV
                    save_to_json(planting_details_list, f"planting-{planting_id}")
                    save_to_csv(planting_details_list, f"planting-{planting_id}")
                
                # To Not Overload The Server
                time.sleep(4)

In [7]:
# GET v2/plantings/{plantingId}/wells.{ext} -> planting-{plantingId}-wells.csv

def func_get_planting_wells():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrive Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Wells
                planting_wells = gr.get_planting_wells(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_wells, int):
                    continue
                else:
                    
                    # For Progress Purposes
                    print(planting_wells)
                    
                    # Save As JSON And CSV
                    save_to_json(planting_wells, f"planting-{planting_id}-wells")
                    save_to_csv(planting_wells, f"planting-{planting_id}-wells")
                
                # To Not Overload The Server
                time.sleep(4)

In [9]:
# GET v2/plantings/{plantingId}/rainfall.{ext} -> planting-{plantingId}-rainfall.csv

def func_get_planting_rainfall():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Details
                rainfall_data = gr.get_planting_rainfall(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(rainfall_data, int):
                    continue
                else:
                    
                    # For CSV Purposes
                    rainfall_data_flat = flatten_json(rainfall_data)
                    rainfall_data_list = list()
                    rainfall_data_list.append(rainfall_data_flat)
                    
                    # For Progress Purposes
                    print(rainfall_data_list)
                    
                    # Save As JSON And CSV
                    save_to_json(rainfall_data_list, f"planting-{planting_id}-rainfall")
                    save_to_csv(rainfall_data_list, f"planting-{planting_id}-rainfall")
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/plantings/{plantingId}/events.{ext} -> planting-{plantingId}-events.csv

def func_get_planting_events():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Details
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                                        
                    # For CSV Purposes
                    planting_events_flat = flatten_json(planting_events)
                    planting_events_list = list()
                    planting_events_list.append(planting_events_flat)
                    
                    # For Progress Purposes
                    print(planting_events_list)
                    
                    # Save As JSON And CSV
                    save_to_json(planting_events_list, f"planting-{planting_id}-events")
                    save_to_csv(planting_events_list, f"planting-{planting_id}-events")
                
                # To Not Overload The Server
                time.sleep(4)

In [10]:
# GET v2/fertilization-events/{id}.{ext} -> fertilization-{fertilization_id}.csv

def func_get_fertilization_event_details():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                    
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is A Fertilization Event, Get ID And Corresponding Event Details
                        if len(event["Fertilization"]) != 0:
                            fertilization_id = event["Fertilization"][0]["Id"]
                            fertilization_event = gr.get_fertilization_event(fertilization_id)
                            fertilization_event_flat = flatten_json(fertilization_event)
                            
                            # For CSV Purposes
                            fertilization_event_list = list()
                            fertilization_event_list.append(fertilization_event_flat)
                            
                            # For Progress Purposes
                            print(fertilization_event_list)
                            
                            # Save As JSON And CSV
                            save_to_json(fertilization_event_list, f"fertilization-{fertilization_id}")
                            save_to_csv(fertilization_event_list, f"fertilization-{fertilization_id}")
                    
                            # To Not Overload The Server
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/fertilization-events/{eventId}/wells.{ext} -> fertilization-{fertilization_id}-wells.csv

def func_get_fertilization_event_wells():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Rnach Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                    
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is A Fertilization Event, Get ID And Corresponding Event Wells
                        if len(event["Fertilization"]) != 0:
                            fertilization_id = event["Fertilization"][0]["Id"]
                            fertilization_event_wells = gr.get_fertilization_event_wells(fertilization_id)
                            
                            # For Progress Purposes
                            print(fertilization_event_wells)
                            
                            # Save As JSON And CSV
                            save_to_json(fertilization_event_wells, f"fertilization-{fertilization_id}-wells")
                            save_to_csv(fertilization_event_wells, f"fertilization-{fertilization_id}-wells")
                    
                            # To Not Overload The Server
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/fertilization-event-wells/{Id}.{ext} - fertilization-{fertilization_id}-well.csv

def func_get_fertilization_event_well():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                    
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is A Fertilization Event, Get ID And Get Corresponding Event Wells
                        if len(event["Fertilization"]) != 0:
                            fertilization_id = event["Fertilization"][0]["Id"]
                            fertilization_event_wells = gr.get_fertilization_event_wells(fertilization_id)
                            
                            # Iterate Through Event Wells And Get ID
                            for well in fertilization_event_wells:
                                fertilization_event_well = gr.get_fertilization_event_well(well["Id"])
                                
                                # For CSV Purposes
                                fertilization_event_well_flat = flatten_json(fertilization_event_well)
                                fertilization_event_well_list = list()
                                fertilization_event_well_list.append(fertilization_event_well_flat)
                            
                                # For Progress Purposes
                                print(fertilization_event_well_list)
                            
                                # Save As JSON And CSV
                                save_to_json(fertilization_event_well_list, f"fertilization-{fertilization_id}-well")
                                save_to_csv(fertilization_event_well_list, f"fertilization-{fertilization_id}-well")
                    
                                # To Not Overload The Server
                                time.sleep(4)
                            
                            # To Not Overload The Server   
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/ranches/{ranchGuid}/fertilizers.{ext} -> {ranchGuid}-fertilizers.csv

def func_get_fertilizers():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Getting Ranch's Fertilizers
        ranch_fertilizers = gr.get_fertilizers(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(ranch_fertilizers, int):
            continue 
        else:
            
            # Flatten Each Fertilizer JSON In List Of Ranch's Fertilizers
            for i in range(0, len(ranch_fertilizers)):
                ranch_fertilizers[i] = flatten_json(ranch_fertilizers[i]) 
            
            # For Progress Purposes
            print(ranch_fertilizers)
            
            # Save As JSON And CSV
            save_to_json(ranch_fertilizers, f"{ranch_guid}-fertilizers")
            save_to_csv(ranch_fertilizers, f"{ranch_guid}-fertilizers")
            
        # To Not Overload The Server
        time.sleep(4)

In [12]:
# GET v2/ranches/{ranchGuid}/fertilizers.{ext} -> {ranchGuid}-fertilizers-available.csv

def func_get_available_fertilizers():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_guid = ranch["Ranch_External_GUID"]
        
        # GET Request For Available Fertilizers Not Associated With Ranch
        available_ranch_fertilizers = gr.get_available_fertilizers(ranch_guid)
        
        # If Result Is A Response Code, Continue
        if isinstance(available_ranch_fertilizers, int):
            continue 
        else:
            
            # Flatten Each Fertilizer JSON In List Of Ranch's Fertilizers
            for i in range(0, len(available_ranch_fertilizers)):
                available_ranch_fertilizers[i] = flatten_json(available_ranch_fertilizers[i]) 
            
            # For Progress Purposes
            print(available_ranch_fertilizers)
            
            # Save As JSON And CSV
            save_to_json(available_ranch_fertilizers, f"{ranch_guid}-fertilizers-=available")
            save_to_csv(available_ranch_fertilizers, f"{ranch_guid}-fertilizers-available")
            
        # To Not Overload The Server
        time.sleep(4)

In [ ]:
# GET v2/irrigation-events/{id}.{ext} -> irrigation-{irrigation_id}.csv

def func_get_irrigation_event_details():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                    
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is An Irrigation Event, Get ID And Corresponding Event Details
                        if event["Irrigation"]:
                            irrigation_id = event["Irrigation"]["Id"]
                            irrigation_event = gr.get_irrigation_event(irrigation_id)
                            irrigation_event_flat = flatten_json(irrigation_event)
                            
                            # For CSV Purposes
                            irrigation_event_list = list()
                            irrigation_event_list.append(irrigation_event_flat)
                            
                            # For Progress Purposes
                            print(irrigation_event_list)
                            
                            # Save As JSON And CSV
                            save_to_json(irrigation_event_list, f"irrigation-{irrigation_id}")
                            save_to_csv(irrigation_event_list, f"irrigation-{irrigation_id}")
                    
                            # To Not Overload The Server
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/irrigation-events/{id}/recommendations-interval-summary.{ext} -> irrigation-{irrigation_id}-summary.csv

def func_get_irrigation_events_recommendations():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Planting Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                    
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is An Irrigation Event, Get ID And Corresponding Recommendation Summary
                        if event["Irrigation"]:
                            irrigation_id = event["Irrigation"]["Id"]
                            irrigation_event_summary = gr.get_cropmanage_irrigation_recommendation(irrigation_id)
                            
                            # For CSV Purposes
                            irrigation_event_summary_list = list()
                            irrigation_event_summary_list.append(irrigation_event_summary)
                            
                            # For Progress Purposes
                            print(irrigation_event_summary_list)
                            
                            # Save As JSON And CSV
                            save_to_json(irrigation_event_summary_list, f"irrigation-{irrigation_id}-summary")
                            save_to_csv(irrigation_event_summary_list, f"irrigation-{irrigation_id}-summary")
                    
                            # To Not Overload The Server
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)

In [ ]:
# GET v2/plantings/{plantingId}/irrigation-events/details.{ext} -> planting-{plantingId}-irrigation.csv

def func_get_irrigation_data():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Plantg In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's Irrigation Data
                irrigation_data = gr.get_irrigation_data(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(irrigation_data, int):
                    continue
                else:
                    
                    # For Progress Purposes
                    print(irrigation_data)
                    
                    # Save As JSON And CSV
                    save_to_json(irrigation_data, f"planting-{planting_id}-irrigation")
                    save_to_csv(irrigation_data, f"planting-{planting_id}-irrigation")
                
                # To Not Overload The Server
                time.sleep(4)

In [3]:
# GET v2/plantings/{plantingId}/et-weather-events.json -> planting-{plantingId}-et-weather-events.csv

def func_get_et_weather_events():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Ranch's ET Weather Events
                et_weather_events = gr.get_et_weather_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(et_weather_events, int):
                    continue
                else:
                
                    # For CSV Purposes
                    et_weather_events_flat = flatten_json(et_weather_events)
                    et_weather_events_list = list()
                    et_weather_events_list.append(et_weather_events_flat)
                    
                    # For Progress Purposes
                    print(et_weather_events_list)
                    
                    # Save As JSON And CSV
                    save_to_json(et_weather_events_list, f"planting-{planting_id}-et-weather-events")
                    save_to_csv(et_weather_events_list, f"planting-{planting_id}-et-weather-events")
                
                # To Not Overload The Server
                time.sleep(4)

In [6]:
# GET v2/tissue-sample-events/{id}.{ext} -> tissue-{id}.csv

def func_get_tissue_sample_event_details():
    
    # Going Through User's Ranches
    for ranch in ranches:
        
        # Using Field To Retrieve Resulting JSON
        ranch_id = ranch["Id"]
        
        # Opening Ranch Plantings JSON File
        with open(f"./json/ranch-{ranch_id}-plantings.json", 'r') as file:
            data = json.load(file)
            
            # Iterating Through Each Planting In The Ranch
            for item in data:
                
                # Using Field To Retrieve Resulting JSON
                planting_id = item["Id"]
                
                # GET Request For Given Planting's Events
                planting_events = gr.get_planting_events(planting_id)
                
                # If Result Is A Response Code, Continue
                if isinstance(planting_events, int):
                    continue
                else:
                
                    # Iterating Through Planting Events
                    for event in planting_events["Events"]:
                        
                        # If There Is A Tissue Sample Event, Get ID And Corresponding Details
                        if len(event["TissueSamples"]) != 0:
                            tissue_id = event["TissueSamples"][0]["Id"]
                            tissue_event = gr.get_tissue_sample_event_details(tissue_id)
                            
                            # For CSV Purposes
                            tissue_event_flat = flatten_json(tissue_event)
                            tissue_event_list = list()
                            tissue_event_list.append(tissue_event_flat)
                            
                            # For Progress Purposes
                            print(tissue_event_list)
                            
                            # Save As JSON And CSV
                            save_to_json(tissue_event_list, f"tissue-{tissue_id}")
                            save_to_csv(tissue_event_list, f"tissue-{tissue_id}")
                        
                            # To Not Overload The Server   
                            time.sleep(4)
                
                # To Not Overload The Server
                time.sleep(4)